# Drug Review Analysis

In [2]:
import pandas as pd
import numpy as np

In [3]:
train_data = pd.read_csv('drugLib_raw/drugLibTrain_raw.tsv', sep='\t', header=0, names=['id', 'drug', 'rating', 'effectiveness', 'side_effects', 'condition', 'benefits_review', 'side_effects_review', 'comments_review'])
train_data.head()

,id,drug,rating,effectiveness,side_effects,condition,benefits_review,side_effects_review,comments_review
0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ..."
1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest..."
2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...
3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...
4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3107 entries, 0 to 3106
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   3107 non-null   int64 
 1   drug                 3107 non-null   object
 2   rating               3107 non-null   int64 
 3   effectiveness        3107 non-null   object
 4   side_effects         3107 non-null   object
 5   condition            3106 non-null   object
 6   benefits_review      3107 non-null   object
 7   side_effects_review  3105 non-null   object
 8   comments_review      3099 non-null   object
dtypes: int64(2), object(7)
memory usage: 218.6+ KB


In [5]:
train_data.drop(columns='id', inplace=True)
train_data.head()

,drug,rating,effectiveness,side_effects,condition,benefits_review,side_effects_review,comments_review
0,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ..."
1,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest..."
2,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...
3,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...
4,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above


In [6]:
train_data.isna().any()

drug                   False
rating                 False
effectiveness          False
side_effects           False
condition               True
benefits_review        False
side_effects_review     True
comments_review         True
dtype: bool

https://aclanthology.org/W10-1205/

In [7]:
train_data.drug.unique().shape

(502,)

(5, 8)

In [19]:
drugs = train_data.groupby(by=['drug']).count()
drugs.drop(columns=['effectiveness', 'side_effects', 'condition', 'benefits_review', 'side_effects_review', 'comments_review'], inplace=True)

In [20]:
drugs.reset_index(inplace=True)
drugs.rename(columns={'rating': 'count'}, inplace=True)

In [21]:
drugs

,drug,count
0,abilify,5
1,accolate,1
2,accupril,1
3,accutane,28
4,aciphex,3
...,...,...
497,zyban,10
498,zyprexa,5
499,zyrtec,23
500,zyrtec-d,3
